In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr
from scipy import stats
from windrose import WindroseAxes

In [ ]:
filepath = "KNMI_Wind.txt" #downloaded from: https://daggegevens.knmi.nl/klimatologie/daggegevens 

# Read the file, skipping everything until the header row
df = pd.read_csv(
    filepath,
    comment='#',       # ignore lines starting with '#'
    skip_blank_lines=True,
    names=["STN", "YYYYMMDD", "DDVEC", "FHVEC"],  # column names
    header=None
)

# Remove metadata before the actual data header
df = df[df["STN"] != "STN"]  # drop the row containing the header string
df = df.apply(lambda col: col.astype(str).str.strip())
df = df.replace({'': pd.NA}) # Replace strings containing only whitespace with NaN

# Convert numeric columns
df["STN"]=pd.to_numeric(df["STN"], errors='coerce') #station number, 330 = Hoek van Holland
df["Date"] = pd.to_datetime(df["YYYYMMDD"], format="%Y%m%d")
df["Year"] = df["Date"].dt.year
df = df.set_index('Date')
df["WindDirection"] = pd.to_numeric(df["DDVEC"], errors='coerce') #degrees
df["WindSpeed"] = pd.to_numeric(df["FHVEC"], errors='coerce') / 10.0  # Convert to m/s
all_years = sorted(df["Year"].unique())

#convert wind speed to Beaufort scale
def wind_speed_to_beaufort(speed_ms):
    """
    Convert wind speed in m/s to Beaufort scale (0-12)
    """
    if pd.isna(speed_ms):
        return np.nan
    elif speed_ms < 0.3:
        return 0
    elif speed_ms < 1.6:
        return 1
    elif speed_ms < 3.4:
        return 2
    elif speed_ms < 5.5:
        return 3
    elif speed_ms < 8.0:
        return 4
    elif speed_ms < 10.8:
        return 5
    elif speed_ms < 13.9:
        return 6
    elif speed_ms < 17.2:
        return 7
    elif speed_ms < 20.8:
        return 8
    elif speed_ms < 24.5:
        return 9
    elif speed_ms < 28.5:
        return 10
    elif speed_ms < 32.7:
        return 11
    else:
        return 12

df['BeaufortScale'] = df['WindSpeed'].apply(wind_speed_to_beaufort).astype('Int64')

print(df.head())

In [ ]:
#plot functions
plt.rcParams["axes.titlesize"] = 12
plt.rcParams['axes.titleweight'] = 'bold'

def plot_monthly_wind_speed_by_month(df, col="WindSpeed"):
    # Ensure index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex")

    fig, axes = plt.subplots(4, 3, figsize=(16, 12))
    axes = axes.flatten()

    # Loop through each month (1 = Jan, ..., 12 = Dec)
    for month in range(1, 13):
        ax = axes[month - 1]

        # Extract only this month's data across all years
        monthly_slice = df[df.index.month == month]

        # Group by year
        summary = monthly_slice.groupby(monthly_slice.index.year)[col].agg(["max", "mean"])

        # Plot
        ax.plot(summary.index, summary['max'], label="Peak", linewidth=2)
        ax.plot(summary.index, summary['mean'], label="Average", linewidth=2)
        ax.set_title(pd.to_datetime(f"2020-{month:02d}-01").strftime("%B"))
        ax.set_xlabel("Year")
        ax.set_ylabel(col)
        ax.grid(True)

        if month == 1:
            ax.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.99])
    plt.suptitle("Monthly average and peaks of daily average wind speeds", fontsize=16, fontweight='bold', y=0.995)
    plt.show()

def plot_histogram(x, title, bins = None, Beaufort = False):
    plt.figure(figsize=(10,5))
    plt.xlabel("Average wind speed (m/s)")
    plt.ylabel("Number of days")
    plt.title(title)
    if bins is not None:
        bins = bins
    else:
        bins = 15
    if Beaufort == True:
        min_val = int(x.min())
        max_val = int(x.max())
        bins = range(min_val, max_val + 2)  # Creates bins for each integer value
        plt.xlabel("Average wind speed (Beaufort Scale)")
        plt.hist(x, bins, rwidth = 0.9, align='left')
        plt.xticks(range(min_val, max_val + 1))  # Center ticks on integers
    else:
        plt.hist(x, bins, rwidth = 0.9, align='left')


In [ ]:
plot_monthly_wind_speed_by_month(df, col="WindSpeed")

plot_histogram(df["BeaufortScale"], 
               title = "Distribution of average daily wind speeds at Hoek van Holland (2001 - 2025)",
               Beaufort = True)

In [ ]:
#plot number of days with wind speed > threshold
threshold = 10 #m/s, or define any other threshold
df_threshold = df[(df["WindSpeed"] >= threshold)]
days_per_year = (
    df_threshold.groupby("Year").size()
    .reindex(all_years, fill_value=0)
)

plt.figure(figsize=(10,5))
plt.bar(days_per_year.index, days_per_year.values, zorder = 3)
plt.xlabel("Year")
plt.ylabel(f"No. of high wind days")
plt.title(f"Count of high-wind days per year (daily average > {threshold} m/s)")
plt.grid(axis ='y', zorder = 0)
plt.show()

In [ ]:
#plot wind rose
def plot_wind_rose(wind_dir, wind_speed, bins, title, max_speed=None, max_date=None):
    new_labels = ["E", "N-E", "N", "N-W", "W", "S-W", "S", "S-E"]
    ax = WindroseAxes.from_ax(theta_labels=new_labels)
    ax.bar(wind_dir, wind_speed, normed=True, bins = bins)
    ax.set_legend(title = 'Wind Speed [m/s]', loc='center right', bbox_to_anchor=(1.25,0.75))
    ax.set_title(title)
     # ---- Add annotation text box ----
    if max_speed is not None and max_date is not None:
        text = (f"Max wind speed: {max_speed} m/s\n"
                f"Date: {max_date}")

        ax.text(0.9, 0.2, text,
                transform=ax.transAxes,
                fontsize=12,
                verticalalignment='bottom',
                horizontalalignment='left',
                bbox=dict(boxstyle="round,pad=0.4", 
                          facecolor="white", 
                          alpha=0.8))

bins_def = np.arange(0, 15, 3) #default bins

plot_wind_rose(df["WindDirection"], df["WindSpeed"], bins=bins_def, title ="Wind speeds and directions at Hoek van Holland Station, 2001-2025")

In [ ]:
#plot daily peaks (only significant winds)
df_significant = df[~df["WindDirection"].between(80, 190)] #exclude winds from E to S

plot_histogram(df_significant["BeaufortScale"],
               title = "Histogram of wind speeds, excluding winds from E to S (2001-2025)",
               Beaufort=True)

In [ ]:
#create dfs for known storms 
def create_storm_dataframes(df, storms_dict):
    """
    storms_dict format: {'storm_name': ('start_date', 'end_date')}
    """
    return {
        name: df[(df.index >= dates[0]) & (df.index <= dates[1])]
        for name, dates in storms_dict.items()
    }

# Define storms
storms = {
    'St_Jude': ('2013-10-24', '2013-11-02'),
    'dirk': ('2013-12-19', '2013-12-29'),
    'ciara': ('2020-02-01', '2020-02-18'),
    'eunice': ('2022-02-15', '2022-02-21'),
    'amy': ('2025-10-02', '2025-10-06'),
    'benjamin': ('2025-10-24', '2025-10-31')
}

storm_dfs = create_storm_dataframes(df, storms)

In [ ]:
#plot wind roses for each storm 
n_storms = len(storm_dfs)
n_cols = 3
n_rows = int(np.ceil(n_storms / n_cols))

fig = plt.figure(figsize=(15, 5 * n_rows))

for idx, (storm_name, storm_df) in enumerate(storm_dfs.items(), 1):
    ax = fig.add_subplot(n_rows, n_cols, idx, projection='windrose')
    new_labels = ["E", "N-E", "N", "N-W", "W", "S-W", "S", "S-E"]
    # ax.set_theta_zero_location('N')
    ax.set_xticklabels(new_labels)
    
    ax.bar(storm_df["WindDirection"], storm_df["WindSpeed"], normed=True, bins=np.arange(0, 16, 1))
    
    start_date = storm_df.index.min().strftime("%d %b %y")
    end_date = storm_df.index.max().strftime("%d %b %y")
    ax.set_title(f"{storm_name.capitalize()}\n({start_date} - {end_date})")
    
    # Add legend to the last plot in each row or the very last plot
    if idx % n_cols == 0 or idx == n_storms:
        ax.legend(title='Wind Speed [m/s]', loc='center right', bbox_to_anchor=(1.3, 0.5))

plt.tight_layout()
plt.show()